In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.maximize_window()

wait = WebDriverWait(driver, 10)

action = ActionChains(driver)

driver.get(
    "https://www.youtube.com/watch?v=zA-F9EtsRJc&list=PLuMuHAJh9g_Py_PSm8gmHdlcil6CQ9QCM"
)

titles = []
playlists = driver.find_element(
    By.XPATH,
    "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/ytd-playlist-panel-renderer/div/div[3]",
)
title_elems = playlists.find_elements(By.ID, "video-title")
# title_elems = title_elems.find_elements(By.XPATH,'')
link_imgs = playlists.find_elements(By.CSS_SELECTOR, "img")
for title in title_elems:
    title = title.text.split("l ")[-1].split(" (with.")
    title, guest = title[0], title[1].split(")")[0] if len(title) != 1 else ""
    title = title.split(" ", 1)
    print(title)
    loc, food = title[0], title[1] if len(title) != 1 else ""
    titles.append([loc, food, guest])

# for link in link_imgs:
#     action.move_to_element(link).perform()
#     link.click()
# wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'img')))

driver.quit()

['부산', '만우장']
['부산', '해진아나고']
['속초', '화진호 이선장네']
['남영동', '옛날감자전']
['반포동', '삼계탕마을']
['한남동', '고다이']
['상봉역', '함평국밥']
['상봉역', '먹거리집']
['담양', '한재골가든']
['아놔', '키 높은 쉐키..']
['하남', '숯불닭갈비']
['광주', '막동이회관']
['구의역', '풍성감자탕']
['종로5가', '효제루']
['속초', '전원식당']
['연희동', '이품']
['매치스성수']
['남영동', '초원']
['청담동', '잔비워, 이남장 삼성점']
['신사동', '가담']
['맘스터치', '슈퍼싸이콤보']
['후암동', '진동세꼬시']
['싱가포르', '현지 맛집 2탄']
['싱가포르', '현지 맛집 1탄']
['흑석시장', '충북순대']
['흑석시장', '진미순대']
['답십리역', '오소록']
['독일', '베를린 특집 2탄']
['독일', '베를린 특집 1탄']
['압구정로데오', '스테이지 바이 고디바']
['명동역', '향미, 개화']
['제주도', '함덕골목 본점']
['제주도', '동복리 해녀촌, 돌다메']
['왕십리역', '굴과찜사랑']
['경동시장', '권영수 대가전골']
['이수역', '돈키돈까스']
['서촌', '도량, 오쁘띠베르']
['하와이', '푸드 & 와인 페스티벌']
['강남', '파이브가이즈']
['정자동', '윤밀원']
['로코스', '비비큐 경리단길본점']
['용산구', '야스노야 본점']
['천안', '청화집']
['북창동', '묵호회집']
['경동시장', '안동집, 김제식당']
['경동시장', '남원통닭']
['의정부', '별미순대국']
['부산', '월드양념통닭']
['공덕동', '원조신촌설렁탕']
['압구정로데오', '와일드버팔로 2탄']
['압구정로데오', '와일드버팔로 1탄']
['익산', '진미식당']
['압구정로데오', '딸랏']
['논현동', '삼호짱뚱이 2탄']
['논현동', '삼호짱뚱이 1탄']
['이태원', 

In [20]:
import pandas as pd

df = pd.DataFrame(titles,columns=['title','food','guest'])
print(df)

    title      food guest
0      부산       만우장      
1      부산     해진아나고      
2      속초  화진호 이선장네      
3     남영동     옛날감자전      
4     반포동     삼계탕마을      
..    ...       ...   ...
147  신미식당       감자탕      
148    화목       순대국      
149    약수       순대국      
150   어머니   대성집 해장국      
151   대성집      도가니탕      

[152 rows x 3 columns]
